Adopted from GDELT Data Wrangle by James Houghton https://nbviewer.jupyter.org/github/JamesPHoughton/Published_Blog_Scripts/blob/master/GDELT%20Wrangler%20-%20Clean.ipynb

Additional GDELT resources: 
    
    GDELT library overview: https://colab.research.google.com/drive/1rnKEHKV1StOwGtFPsCctKDPTBB_kHOc_?usp=sharing 
    
    GDELT with big data: https://github.com/linwoodc3/gdeltPyR/wiki/Pulling-Large-GDELT-Data
        

# PART I: Get GDELT DATA FOR NIGER


### Get the GDELT index files

In [1]:
import requests
import lxml.html as lh

gdelt_base_url = 'http://data.gdeltproject.org/events/'

# get the list of all the links on the gdelt file page
page = requests.get(gdelt_base_url+'index.html') #Grab GDELT reference list which is by day
doc = lh.fromstring(page.content)
link_list = doc.xpath("//*/ul/li/a/@href") #Returns all the possible CSV files of GDELT data as a references list

# separate out those links that begin with four digits 
'''
Will extract just the days resulting in list like: 
['20200617.export.CSV.zip',
 '20200616.export.CSV.zip',
 '20200615.export.CSV.zip',...]
 Until 2015
'''

file_list = [x for x in link_list if str.isdigit(x[0:4])]
file_list

['20201001.export.CSV.zip',
 '20200930.export.CSV.zip',
 '20200929.export.CSV.zip',
 '20200928.export.CSV.zip',
 '20200927.export.CSV.zip',
 '20200926.export.CSV.zip',
 '20200925.export.CSV.zip',
 '20200924.export.CSV.zip',
 '20200923.export.CSV.zip',
 '20200922.export.CSV.zip',
 '20200921.export.CSV.zip',
 '20200920.export.CSV.zip',
 '20200919.export.CSV.zip',
 '20200918.export.CSV.zip',
 '20200917.export.CSV.zip',
 '20200916.export.CSV.zip',
 '20200915.export.CSV.zip',
 '20200914.export.CSV.zip',
 '20200913.export.CSV.zip',
 '20200912.export.CSV.zip',
 '20200911.export.CSV.zip',
 '20200910.export.CSV.zip',
 '20200909.export.CSV.zip',
 '20200908.export.CSV.zip',
 '20200907.export.CSV.zip',
 '20200906.export.CSV.zip',
 '20200905.export.CSV.zip',
 '20200904.export.CSV.zip',
 '20200903.export.CSV.zip',
 '20200902.export.CSV.zip',
 '20200901.export.CSV.zip',
 '20200831.export.CSV.zip',
 '20200830.export.CSV.zip',
 '20200829.export.CSV.zip',
 '20200828.export.CSV.zip',
 '20200827.export.CS

In [2]:
#Counters to help assess how many files are coming and going out
infilecounter = 0
outfilecounter = 0

### Uses GDELT Index file list to download GDELT data for that day for that country

In [3]:
import os.path #To help navigate the file directories
import urllib #To request from GDELT
import zipfile #TO unzip the files we downlaod
import glob #To go through multiple files in a directory
import operator 

local_path = './results/' # Will save to empy results folder to help keep file clean

fips_country_code = 'NG'  ## !!!!! THIS IS THE NIGER COUNTRY CODE GETS ONLY NIGER DATA!!!!

#Adjust list number to get days wanted 
for compressed_file in file_list[:7]: #!!!!!Only getting index 0 to 6!!!!!!
    print(compressed_file,)
    
    # if we dont have the compressed file stored locally, go get it. Keep trying if necessary.
    while not os.path.isfile(local_path+compressed_file): 
        print('downloading,'),
        urllib.request.urlretrieve(url=gdelt_base_url+compressed_file, 
                           filename=local_path+compressed_file)
        
    # extract the contents of the compressed file to a temporary directory    
    print('extracting,'),
    z = zipfile.ZipFile(file=local_path+compressed_file, mode='r')    
    z.extractall(path=local_path+'tmp/')
    
    # parse each of the csv files in the working directory, 
    print('parsing,'),
    for infile_name in glob.glob(local_path+'tmp/*'):
        outfile_name = local_path+fips_country_code+'%04i.tsv'%outfilecounter
        
        # open the infile and outfile
        with open(infile_name, mode='r', encoding="ISO-8859-1") as infile, open(outfile_name, mode='w') as outfile:
            for line in infile:
                # extract lines with our interest country code
                if fips_country_code in operator.itemgetter(51, 37, 44)(line.split('\t')):    
                    outfile.write(line)
            outfilecounter +=1
            
        # delete the temporary file
        os.remove(infile_name)
    infilecounter +=1
    print('done', infilecounter)
    

20201001.export.CSV.zip
downloading,
extracting,
parsing,
done 1
20200930.export.CSV.zip
downloading,
extracting,
parsing,
done 2
20200929.export.CSV.zip
downloading,
extracting,
parsing,
done 3
20200928.export.CSV.zip
extracting,
parsing,
done 4
20200927.export.CSV.zip
extracting,
parsing,
done 5
20200926.export.CSV.zip
extracting,
parsing,
done 6
20200925.export.CSV.zip
extracting,
parsing,
done 7


# PART II:  PARSE DATA AGAIN

### Read in the data

In [4]:
import pandas as pd

# Get the GDELT field names from a helper file
colnames = pd.read_csv('CSV.header.fieldids.csv')['Field Name']


# Build DataFrames from each of the intermediary files
files = glob.glob(local_path+fips_country_code+'*')
DFlist = []
for active_file in files:
    print(active_file)
    DFlist.append(pd.read_csv(active_file, sep='\t', header=None, dtype=str,
                              names=colnames, index_col=['GLOBALEVENTID'], encoding='iso-8859-1'))

# Merge the file-based dataframes and save a pickle
DF = pd.concat(DFlist)
DF.to_pickle(local_path+'backup'+fips_country_code+'.pickle')    
    
# once everythin is safely stored away, remove the temporary files
for active_file in files:
    os.remove(active_file)

./results\NG0000.tsv
./results\NG0001.tsv
./results\NG0002.tsv
./results\NG0003.tsv
./results\NG0004.tsv
./results\NG0005.tsv
./results\NG0006.tsv


In [5]:
import pickle

Niger_Data = pd.read_pickle(r"./results/backupNG.pickle")

### See top 5 lines of data

In [6]:
Niger_Data.head()

,SQLDATE,MonthYear,Year,FractionDate,Actor1Code,Actor1Name,Actor1CountryCode,Actor1KnownGroupCode,Actor1EthnicCode,Actor1Religion1Code,...,Actor2Geo_FeatureID,ActionGeo_Type,ActionGeo_FullName,ActionGeo_CountryCode,ActionGeo_ADM1Code,ActionGeo_Lat,ActionGeo_Long,ActionGeo_FeatureID,DATEADDED,SOURCEURL
GLOBALEVENTID,,,,,,,,,,,,,,,,,,,,,
949581911,20201001,202010,2020,2020.7425,NaN,NaN,NaN,NaN,NaN,NaN,...,NG,1,Niger,NG,NG,16,8,NG,20201001,https://thenationonlineng.net/robbers-invade-s...
949581913,20201001,202010,2020,2020.7425,NaN,NaN,NaN,NaN,NaN,NaN,...,NG,1,Niger,NG,NG,16,8,NG,20201001,https://thenationonlineng.net/robbers-invade-s...
949582054,20201001,202010,2020,2020.7425,NaN,NaN,NaN,NaN,NaN,NaN,...,NG,1,Niger,NG,NG,16,8,NG,20201001,https://thenationonlineng.net/robbers-invade-s...
949583634,20201001,202010,2020,2020.7425,HLH,HOSPITAL,NaN,NaN,NaN,NaN,...,NaN,1,Niger,NG,NG,16,8,NG,20201001,https://thenationonlineng.net/robbers-invade-s...
949587178,20201001,202010,2020,2020.7425,GOV,GOVERNMENT,NaN,NaN,NaN,NaN,...,NaN,1,Niger,NG,NG,16,8,NG,20201001,https://naija247news.com/2020/09/28/niger-offi...


In [7]:
Niger_Data.columns

Index(['SQLDATE', 'MonthYear', 'Year', 'FractionDate', 'Actor1Code',
       'Actor1Name', 'Actor1CountryCode', 'Actor1KnownGroupCode',
       'Actor1EthnicCode', 'Actor1Religion1Code', 'Actor1Religion2Code',
       'Actor1Type1Code', 'Actor1Type2Code', 'Actor1Type3Code', 'Actor2Code',
       'Actor2Name', 'Actor2CountryCode', 'Actor2KnownGroupCode',
       'Actor2EthnicCode', 'Actor2Religion1Code', 'Actor2Religion2Code',
       'Actor2Type1Code', 'Actor2Type2Code', 'Actor2Type3Code', 'IsRootEvent',
       'EventCode', 'EventBaseCode', 'EventRootCode', 'QuadClass',
       'GoldsteinScale', 'NumMentions', 'NumSources', 'NumArticles', 'AvgTone',
       'Actor1Geo_Type', 'Actor1Geo_FullName', 'Actor1Geo_CountryCode',
       'Actor1Geo_ADM1Code', 'Actor1Geo_Lat', 'Actor1Geo_Long',
       'Actor1Geo_FeatureID', 'Actor2Geo_Type', 'Actor2Geo_FullName',
       'Actor2Geo_CountryCode', 'Actor2Geo_ADM1Code', 'Actor2Geo_Lat',
       'Actor2Geo_Long', 'Actor2Geo_FeatureID', 'ActionGeo_Type',
      

### Helper Function  to turn codebooks  into look up tables

In [8]:
def ref_dict(df):
    cols = list(df)
    ref_dict = {}
    for row in df.iterrows(): 
        ref_dict[row[1][cols[0]]] = row[1][cols[1]]
    
    return ref_dict

### Convert each codebook and store in object

In [10]:
#Read in event codes
eventCodes = ref_dict(pd.read_csv("./Ref Codes/CAMEO.eventcodes.txt", sep='\t'))
#Read in Goldsteinscale
goldScale = ref_dict(pd.read_csv("./Ref Codes/CAMEO.goldsteinscale.txt", sep='\t'))
#Read in ethnic groups
ethnicCodes =ref_dict(pd.read_csv("./Ref Codes/CAMEO.ethnic.txt", sep='\t'))
#Read in known Groups
knownGroups = ref_dict(pd.read_csv("./Ref Codes/CAMEO.knowngroup.txt", sep='\t'))
#Read in relgion
religionCodes = ref_dict(pd.read_csv("./Ref Codes/CAMEO.religion.txt", sep='\t'))
#Read in type
typeCodes = ref_dict(pd.read_csv("./Ref Codes/CAMEO.type.txt", sep='\t'))

religionCodes

{'ADR': 'African Diasporic Religion',
 'ALE': 'Alewi',
 'ATH': 'Agnostic',
 'BAH': 'Bahai Faith',
 'BUD': 'Buddhism',
 'CHR': 'Christianity',
 'CON': 'Confucianism',
 'CPT': 'Coptic',
 'CTH': 'Catholic',
 'DOX': 'Orthodox',
 'DRZ': 'Druze',
 'HIN': 'Hinduism',
 'HSD': 'Hasidic',
 'ITR': 'Indigenous Tribal Religion',
 'JAN': 'Jainism',
 'JEW': 'Judaism',
 'JHW': "Jehovah's Witness",
 'LDS': 'Latter Day Saints',
 'MOS': 'Muslim',
 'MRN': 'Maronite',
 'NRM': 'New Religious Movement',
 'PAG': 'Pagan',
 'PRO': 'Protestant',
 'SFI': 'Sufi',
 'SHI': 'Shia',
 'SHN': 'Old Shinto School',
 'SIK': 'Sikh',
 'SUN': 'Sunni',
 'TAO': 'Taoist',
 'UDX': 'Ultra-Orthodox',
 'ZRO': 'Zoroastrianism'}

In [11]:
# Turn colnames into list for ref

cross_ref = list(colnames)


In [12]:
# Create look up table to get values instead of numbers

look_up_code = {"eventCodes": [26,27,28], "goldScale":[30], "ethnicCodes":[9,19], "knownGroups":[8,18], 
                "religionCodes":[10,11,20,21], "typeCodes":[12,13,14,22,23,24]}

In [16]:
'''
Helper function to user can reorient data based on interest from codes

data: Niger_Data - pandas dataframe
ref: key value from look_look_code - string
codebook: reference 
'''

import math

def search_dict(data,ref, codebook):
    res = {}
    look_up = look_up_code[ref]
    print(look_up)
    col_names = []
    for i in look_up: 
        col_names.append(cross_ref[i])
    print (col_names)
    
    for col in col_names: 
        for row in data.iterrows(): 
            if isinstance(row[1][col],float):
                #print (type(row[1][col]), col)
                pass
            else: 
                #print (col)
                var = codebook[row[1][col]].upper()
                #print (var, row[1][col])
                if var in res.keys(): 
                    #print(row[1][col])
                    res[var].append(dict(row[1]))
                else: 
                    res[var] = [dict(row[1])]
    return res
    


In [17]:
res = search_dict(Niger_Data, "religionCodes", religionCodes)
res.keys()

[10, 11, 20, 21]
['Actor1Religion1Code', 'Actor1Religion2Code', 'Actor2Religion1Code', 'Actor2Religion2Code']


dict_keys(['CHRISTIANITY', 'MUSLIM', 'CATHOLIC'])

In [18]:
#verfication to ensure code is working properly
for k,v in res.items(): 
    print (k, ": ", len(v))

CHRISTIANITY :  3
MUSLIM :  10
CATHOLIC :  1


In [19]:
#Put each collection of articles in a Dataframe
list_res = []

for cat in res.values(): 
    #print(cat)
    list_res.append(pd.DataFrame(cat))
    

In [20]:
list_res[0] #access the group you are interested in by changing the variables

,SQLDATE,MonthYear,Year,FractionDate,Actor1Code,Actor1Name,Actor1CountryCode,Actor1KnownGroupCode,Actor1EthnicCode,Actor1Religion1Code,...,Actor2Geo_FeatureID,ActionGeo_Type,ActionGeo_FullName,ActionGeo_CountryCode,ActionGeo_ADM1Code,ActionGeo_Lat,ActionGeo_Long,ActionGeo_FeatureID,DATEADDED,SOURCEURL
0,20201001,202010,2020,2020.7425,CHRCTH,CATHOLIC,NaN,NaN,NaN,CHR,...,NaN,1,Niger,NG,NG,16,8,NG,20201001,https://www.catholicworldreport.com/2020/10/01...
1,20200926,202009,2020,2020.7288,CHR,CHRISTIAN,NaN,NaN,NaN,CHR,...,-2014610,1,Niger,NG,NG,16,8,NG,20200926,https://punchng.com/clark-nwodo-others-berate-...
2,20201001,202010,2020,2020.7425,NaN,NaN,NaN,NaN,NaN,NaN,...,-1086069,4,"Tahoua, Tahoua, Niger",NG,NG06,14.8888,5.2692,-1086069,20201001,https://www.wvi.org/stories/niger/intensifying...


### Homework 4: Do some type of analysis with GDELT data. It can be country focused (e.g. Guatemala) or topic focused (e.g. attacks or bilateral agreements)

### Must write in the first cell what you are interested in. Code must work but results can be garabage. Update the GDELT parameters to get the information you want and then include some type of plot can be a graph or can be a map.  

### Total Points Possible 19
      